In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
games = pd.read_csv('games.csv')
players = pd.read_csv('players.csv')
plays = pd.read_csv('plays.csv')
tackles = pd.read_csv('tackles.csv')
tracking1 = pd.read_csv('tracking_week_1.csv')
tracking2 = pd.read_csv('tracking_week_2.csv')
tracking3 = pd.read_csv('tracking_week_3.csv')
tracking4 = pd.read_csv('tracking_week_4.csv')
tracking5 = pd.read_csv('tracking_week_5.csv')
tracking6 = pd.read_csv('tracking_week_6.csv')
tracking7 = pd.read_csv('tracking_week_7.csv')
tracking8 = pd.read_csv('tracking_week_8.csv')
tracking9 = pd.read_csv('tracking_week_9.csv')

In [3]:
tracking = pd.concat([tracking1, tracking2, tracking3, tracking4, tracking5, tracking6, tracking7, tracking8, tracking9])
tracking['event'] = tracking['event'].fillna(0)
carriers = pd.merge(plays, tracking, how='left', left_on=['gameId', 'playId', 'ballCarrierId'] ,right_on=['gameId', 'playId', 'nflId'])
df = pd.merge(tracking, plays.loc[:, ['gameId', 'playId', 'ballCarrierId']], how='left', left_on=['gameId', 'playId', 'nflId'] ,right_on=['gameId', 'playId', 'ballCarrierId'])
df = pd.merge(df, tackles.loc[(tackles['tackle']==1)|(tackles['assist']==1)|(tackles['forcedFumble']==1)|(tackles['pff_missedTackle']==1), ['gameId', 'playId', 'nflId', 'tackle', 'assist', 'forcedFumble', 'pff_missedTackle']], how='left', on=['gameId', 'playId', 'nflId'])

In [4]:
df.rename(columns={'ballCarrierId': 'isCarrier', 'tackle': 'isTackler', 'assist': 'isAssistant'}, inplace=True)
df['isCarrier'] = df['isCarrier'].fillna(0)
df['isTackler'] = df['isTackler'].fillna(0)
df['isAssistant'] = df['isAssistant'].fillna(0)
df['forcedFumble'] = df['forcedFumble'].fillna(0)
df['pff_missedTackle'] = df['pff_missedTackle'].fillna(0)

In [5]:
df['gameId'].unique()

array([2022090800, 2022091100, 2022091101, 2022091102, 2022091103,
       2022091104, 2022091105, 2022091106, 2022091107, 2022091108,
       2022091109, 2022091110, 2022091111, 2022091112, 2022091113,
       2022091200, 2022091500, 2022091800, 2022091801, 2022091802,
       2022091803, 2022091804, 2022091805, 2022091806, 2022091807,
       2022091808, 2022091809, 2022091810, 2022091811, 2022091812,
       2022091900, 2022091901, 2022092200, 2022092500, 2022092501,
       2022092502, 2022092503, 2022092504, 2022092505, 2022092506,
       2022092507, 2022092508, 2022092509, 2022092510, 2022092511,
       2022092512, 2022092513, 2022092600, 2022092900, 2022100200,
       2022100201, 2022100202, 2022100203, 2022100204, 2022100205,
       2022100206, 2022100207, 2022100208, 2022100209, 2022100210,
       2022100211, 2022100212, 2022100213, 2022100300, 2022100600,
       2022100900, 2022100901, 2022100902, 2022100903, 2022100904,
       2022100905, 2022100906, 2022100907, 2022100908, 2022100

In [6]:
df.drop(df[(df['gameId']==2022091801) & (df['playId']==63)].index, inplace=True)
df.drop(df[(df['gameId']==2022101605) & (df['playId']==2970)].index, inplace=True)
df.drop(df[(df['gameId']==2022102307)&(df['playId']==1505)].index, inplace=True)

In [13]:
grouped = df.groupby(['gameId', 'playId'])

frames = []

for (game, play), group in grouped:
    print(game, play)
    n = len(group) / 23  # number of frames
    for i in range(1, int(n) + 1):
        frame = group[group['frameId'] == i]
        ball_info = frame[frame['club'] == 'football'][['x', 'y']].iloc[0]
        carrier_info = frame[frame['isCarrier']!=0][['x', 'y', 'o', 'dir', 's']].iloc[0]

        frame.loc[:, 'footballDistance'] = np.sqrt((frame['x'] - ball_info['x'])**2 + (frame['y'] - ball_info['y'])**2)
        frame.loc[:, 'carrierDistance'] = np.sqrt((frame['x'] - carrier_info['x'])**2 + (frame['y'] - carrier_info['y'])**2)
        frame.loc[:, 'orientationDifference'] = abs(frame['o'] - carrier_info['o'])
        frame.loc[:, 'motionAngleDifference'] = abs(frame['dir'] - carrier_info['dir'])
        frame.loc[:, 'speedDifference'] = frame['s'] - carrier_info['s']
        frames.append(frame)

new_df = pd.concat(frames, ignore_index=True)

2022090800 56
2022090800 80
2022090800 101
2022090800 122
2022090800 146
2022090800 167
2022090800 191
2022090800 212
2022090800 236
2022090800 299
2022090800 343
2022090800 364
2022090800 393
2022090800 414
2022090800 438
2022090800 486
2022090800 529
2022090800 569
2022090800 593
2022090800 617
2022090800 646
2022090800 692
2022090800 775
2022090800 818
2022090800 867
2022090800 896
2022090800 933
2022090800 980
2022090800 1030
2022090800 1102
2022090800 1187
2022090800 1230
2022090800 1254
2022090800 1286
2022090800 1334
2022090800 1358
2022090800 1385
2022090800 1406
2022090800 1563
2022090800 1712
2022090800 1736
2022090800 1757
2022090800 1836
2022090800 1946
2022090800 1967
2022090800 2043
2022090800 2072
2022090800 2093
2022090800 2137
2022090800 2163
2022090800 2184
2022090800 2208
2022090800 2232
2022090800 2307
2022090800 2336
2022090800 2360
2022090800 2464
2022090800 2485
2022090800 2506
2022090800 2527
2022090800 2551
2022090800 2572
2022090800 2599
2022090800 2648
202209

"new_df = pd.DataFrame()\n\nunique_games = df['gameId'].unique()\nfor game in unique_games:\n    unique_plays = df[df['gameId']==game]['playId'].unique()\n    for play in unique_plays:\n        n = len(df[(df['gameId']==game)&(df['playId']==play)])/23 #number of frames\n        for i in range(1, int(n)+1):\n            frame = df[(df['gameId']==game)&(df['playId']==play)&(df['frameId']==i)]\n            ball_x = frame.loc[frame['club']=='football', 'x'].values[0]\n            ball_y = frame.loc[frame['club']=='football', 'y'].values[0]\n            carrier_x = frame.loc[frame['isCarrier'].isna()==False, 'x'].values[0]\n            carrier_y = frame.loc[frame['isCarrier'].isna()==False, 'y'].values[0]\n            frame['footballDistance'] = np.sqrt((frame['x']-ball_x)**2 + (frame['y']-ball_y)**2)\n            frame['carrierDistance'] = np.sqrt((frame['x']-carrier_x)**2 + (frame['y']-carrier_y)**2)\n            new_df = pd.concat([new_df, frame])    "

In [16]:
new_df.to_csv('new_df.csv')